In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

In [ ]:
# Clone
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# Configure
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile

In [ ]:
# Compile
!make

In [ ]:
# Make a copy of yolov4.cfg
!cp cfg/yolov4-tiny-custom.cfg cfg/yolov4_tiny_training.cfg

In [ ]:
# Change lines in yolov4.cfg file
!sed -i 's/batch=1/batch=64/' cfg/yolov4_training.cfg
!sed -i 's/subdivisions=1/subdivisions=32/' cfg/yolov4_training.cfg
!sed -i 's/width=608/width=416/' cfg/yolov4_training.cfg
!sed -i 's/height=608/height=416/' cfg/yolov4_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov4_training.cfg
!sed -i 's/steps=400000,450000/steps=3200,3600/' cfg/yolov4_training.cfg
!sed -i '220 s@classes=80@classes=1@' cfg/yolov4_training.cfg
!sed -i '269 s@classes=80@classes=1@' cfg/yolov4_training.cfg
!sed -i '212 s@filters=255@filters=18@' cfg/yolov4_training.cfg
!sed -i '263 s@filters=255@filters=18@' cfg/yolov4_training.cfg

In [ ]:
#Defines the number of classes and their name
!echo -e 'Acidente' > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov4' > data/obj.data

In [ ]:
#Copy files to your drive
!cp cfg/yolov4_tiny_training.cfg /mydrive/yolov4/yolov4_tiny_testing.cfg
!cp data/obj.names /mydrive/yolov4/classes.txt

In [ ]:
#Unzip the folder containing the images and labels
!mkdir data/obj
!unzip /mydrive/yolov4/images.zip -d data/obj

In [ ]:
#Collect only jpg files and save their path
import glob
images_list = glob.glob("data/obj/*.jpg")
with open("data/train.txt", "w") as f:
    f.write("\n".join(images_list))

In [ ]:
#Copy the file responsible for training the model
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29

In [ ]:
#Start training
!./darknet detector train data/obj.data cfg/yolov4_tiny_training.cfg yolov4.conv.137 -dont_show
# Uncomment below and comment above to re-start your training from last saved weights
#!./darknet detector train data/obj.data cfg/yolov4_training.cfg /mydrive/yolov4/yolov4_training_last.weights -dont_show

In [ ]:
#Test your model using darknet and saving the output to your drive
!./darknet detector demo data/obj.data /mydrive/yolov4/yolov4_tiny_testing.cfg /mydrive/yolov4/yolov4_tiny_training_last.weights -dont_show /mydrive/yolov4/any_file.mp4 -thresh 0.2 -i 0 -out_filename /mydrive/yolov4/final_result.mp4